## MNIST 테스트 및 미니배치 학습 해보기

In [3]:
import numpy as np
import matplotlib.pylab as plt
import sys, os, pickle
from PIL import Image
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_text, t_test) = \
    load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(t_train.shape)
print(x_text.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [4]:
def sigmoid(x):
    y = 1 / (1 + np.exp(-x))
    return y

In [5]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

In [6]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [7]:
img = x_train[3] # img는 0번째 이미지의 행렬을 의미
label = t_train[3] # label은 0번째 이미지의 숫자값을 의미
print(label)

1


In [8]:
print(img.shape)
img = img.reshape(28, 28)
print(img.shape)
img_show(img)

(784,)
(28, 28)


# MNIST를 신경망을 이용하여 추론해보기

In [8]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

In [9]:
def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


In [10]:
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)


    return y

In [11]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i]) # 이미 정의된 피라미터 network와 i번째 이미지의 행렬값을 보내준다.
    p= np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p == t[i]: # 가장 높은 확률의 숫자가 현재 정답과 일치하면 카운트를 한다
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


### 실제 과정을 수동으로 계산하기

In [12]:
print("x : ", x[6].shape)
print()
print("W1 : ", network['W1'].shape)
print("W2 : ", network['W2'].shape)
print("W3 : ", network['W3'].shape)
print()
print("b1 : ", network['b1'].shape)
print("b2 : ", network['b2'].shape)
print("b3 : ", network['b3'].shape)
print()

# 입력층 -> 은닉층
print("입력층 -> 은닉층")
print(x[6].shape, " * ", network['W1'].shape)

# 은닉층1 -> 은닉층 2
print("\n은닉층1 -> 은닉층 2")
r = np.dot(x[6],network['W1']) +  network['b1']
r = sigmoid(r)
print(r.shape, " * ", network['W2'].shape)

# 은닉층2 -> 은닉층3
print("\n은닉층2 -> 은닉층3")
r2 = np.dot(r,network['W2']) +  network['b2']
r2 = sigmoid(r2)
print(r2.shape, " * ", network['W3'].shape)

# 은닉층3 -> 출력층
print("\n은닉층3 -> 출력층")
r3 = np.dot(r2,network['W3']) +  network['b3']
r3 = sigmoid(r3)
print(r3.shape)
print(np.sum(r3))
print(r3)

print("\n계산 결과 추론된 숫자 : ", np.argmax(r3))

label = t[6] # label은 0번째 이미지의 숫자값을 의미
print("실제 정답 : ", label)

x :  (784,)

W1 :  (784, 50)
W2 :  (50, 100)
W3 :  (100, 10)

b1 :  (50,)
b2 :  (100,)
b3 :  (10,)

입력층 -> 은닉층
(784,)  *  (784, 50)

은닉층1 -> 은닉층 2
(50,)  *  (50, 100)

은닉층2 -> 은닉층3
(100,)  *  (100, 10)

은닉층3 -> 출력층
(10,)
4.722437
[0.00425427 0.04815251 0.0219804  0.1389664  0.99908686 0.90706533
 0.39322525 0.43514013 0.8571234  0.9174423 ]

계산 결과 추론된 숫자 :  4
실제 정답 :  4


## 배치 크기를 늘려 계산하는 방식
- 일반적으로 작은 배열을 여러번 계산하는 것 보다
- 큰 배열을 한번에 계산하는게 컴퓨터 연산에 더 이점이 있다

In [13]:
x, t = get_data()
network = init_network()
batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("\nAccuracy:" + str(float(accuracy_cnt) / len(x)))


Accuracy:0.9352


### 전체를 한번에 계산 (행렬의 곱이 가능하게 피연산자의 행, 열의 개수를 맞추면 계산이 가능)

In [14]:
x, t = get_data()
network = init_network()
accuracy_cnt = 0

x_batch = x
y_batch = predict(network, x_batch)
p = np.argmax(y_batch, axis=1)
accuracy_cnt += np.sum(p == t)

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
